In [1]:
import numpy as np
import itertools
import csv


K.<w> = CyclotomicField(3)
F.<f> = CyclotomicField(6)
M9 = MatrixSpace(K, 9,9)
M36 = MatrixSpace(K, 36,36)
M27 = MatrixSpace(K, 27,27)

V4 = MatrixSpace(K, 4,1)
V3 = MatrixSpace(K, 3,1)
V9= MatrixSpace(K, 9,1)

In [2]:
# define chinese remainder isomorphism

def chinese_rem(x):
    return [mod(x,3), mod(x,2)]


chinese_rem(2)



[2, 0]

In [3]:
# define inverse chinese remainder isomorphism

def inv_chinese_rem(k,x):
    return mod(4*k + 3*x,6)

inv_chinese_rem(2,0)

2

In [4]:
iteration_list = []
for a1 in range(0,6):
    for a2 in range(0,6):
        k = chinese_rem(a1)[0]
        l = chinese_rem(a2)[0]
        x = chinese_rem(a1)[1]
        y = chinese_rem(a2)[1]
        iteration_list.append([k,l,x,y])
                
len(iteration_list)

36

In [5]:
# Define qudit basis

def unit_vec(i,d):
    vec = np.zeros(d)
    for l in range(0,d):
        if l == i:
            vec[l] = 1
    return vec.transpose()

V9(vector(unit_vec(1,9)))

[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [6]:
# define Bell basis

def Bell_basis(d):
    K.<w> = CyclotomicField(d)
    V = MatrixSpace(K, 1, d**2)
    L = []
    for i in range(0,d):
        L.append(vector(np.kron(unit_vec(i,d),unit_vec(i,d))))
    return V(sum(L))

    
    
print(Bell_basis(6))

[1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1]


In [7]:
# define Z gate

def Z_gate(d):
    K.<w> = CyclotomicField(d)
    M = MatrixSpace(K, d,d)
    L = []
    for i in range(0,d):
        L.append(w**i)   
    return M(diagonal_matrix(L))

Z_gate(6)

[     1      0      0      0      0      0]
[     0      w      0      0      0      0]
[     0      0  w - 1      0      0      0]
[     0      0      0     -1      0      0]
[     0      0      0      0     -w      0]
[     0      0      0      0      0 -w + 1]

In [11]:
# define X gate

def X_gate(d):
    K.<w> = CyclotomicField(d)
    M = MatrixSpace(K, d,d)
    L = np.zeros((d,d))
    for j in range(0,d):
         L[(j+1) % d, j] = 1
    return M(L)

X_gate(6)

[0 0 0 0 0 1]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]

In [13]:
# define WH operators

def powers_of_X(d):
    output = []
    for i in range(0,d):
        output.append(X_gate(d)**i)
    return output

def powers_of_X_neg(d):
    output = []
    for i in range(0,d):
        output.append(X_gate(d)**(-i))
    return output

def powers_of_Z(d):
    output = []
    for i in range(0,d):
        output.append(Z_gate(d)**i)
    return output

# test 

powers_of_X(6) == [X_gate(6)**0, X_gate(6),  X_gate(6)**2, X_gate(6)**3,  X_gate(6)**4,  X_gate(6)**5] and powers_of_Z(6) == [Z_gate(6)**0, Z_gate(6), Z_gate(6)**2, Z_gate(6)**3, Z_gate(6)**4, Z_gate(6)**5]

True

In [14]:
# define WH basis 

def WH_basis(a,b,d):
    return ((powers_of_Z(d)[a]*(powers_of_X_neg(d)[b])).tensor_product(X_gate(d)**0))*(Bell_basis(d)).transpose()

# test 

matrix(np.array(WH_basis(3,1,6)).reshape((6,6))) == powers_of_Z(6)[3]*(powers_of_X_neg(6)[1])


True

In [15]:
# define Fourier gate

def Fourier_gate(d):
    K.<w> = CyclotomicField(d)
    M = MatrixSpace(K, d,d)
    L = []
    for i in range(0,d):
        for j in range(0,d):
            L.append(w**(i*j))
    return M(np.array(L).reshape((d,d)))
    


# test

Fourier_gate(6)


[     1      1      1      1      1      1]
[     1      w  w - 1     -1     -w -w + 1]
[     1  w - 1     -w      1  w - 1     -w]
[     1     -1      1     -1      1     -1]
[     1     -w  w - 1      1     -w  w - 1]
[     1 -w + 1     -w     -1  w - 1      w]

In [16]:
# define controlled X gate

def CX_gate(d):
    K.<w> = CyclotomicField(d)
    M = MatrixSpace(K, d**2,d**2)
    V = MatrixSpace(K, d, d)
    L = []
    for i in range(0,d):
        L.append((V(vector(unit_vec(i,d)).tensor_product(vector(unit_vec(i,d))))).tensor_product(X_gate(d)**i))
        #L.append((X_gate(d)**i).tensor_product(V(vector(unit_vec(i,d)).tensor_product(vector(unit_vec(i,d).transpose())))))
    return M(sum(L))

# test

CX_gate(3)

[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0]

In [17]:
# define controlled Z gate

def CZ_gate(d):
    K.<w> = CyclotomicField(d)
    M = MatrixSpace(K, d**2,d**2)
    V = MatrixSpace(K, d, d)
    L = []
    for i in range(0,d):
        L.append((V(vector(unit_vec(i,d)).tensor_product(vector(unit_vec(i,d))))).tensor_product(Z_gate(d)**i))
        #L.append((Z_gate(d)**i).tensor_product(V(vector(unit_vec(i,d)).tensor_product(vector(unit_vec(i,d))))))
    return M(sum(L))

# test

CZ_gate(3)

[     1      0      0      0      0      0      0      0      0]
[     0      1      0      0      0      0      0      0      0]
[     0      0      1      0      0      0      0      0      0]
[     0      0      0      1      0      0      0      0      0]
[     0      0      0      0      w      0      0      0      0]
[     0      0      0      0      0 -w - 1      0      0      0]
[     0      0      0      0      0      0      1      0      0]
[     0      0      0      0      0      0      0 -w - 1      0]
[     0      0      0      0      0      0      0      0      w]

In [18]:
# define unitary that maps to WH basis

def U_WH(d):
    return CX_gate(d)*((Fourier_gate(d)).tensor_product(X_gate(d)**0))

U_WH(3)

[     1      0      0      1      0      0      1      0      0]
[     0      1      0      0      1      0      0      1      0]
[     0      0      1      0      0      1      0      0      1]
[     0      0      1      0      0      w      0      0 -w - 1]
[     1      0      0      w      0      0 -w - 1      0      0]
[     0      1      0      0      w      0      0 -w - 1      0]
[     0      1      0      0 -w - 1      0      0      w      0]
[     0      0      1      0      0 -w - 1      0      0      w]
[     1      0      0 -w - 1      0      0      w      0      0]

In [19]:
# test

U_WH(3)*V9(vector(np.kron(unit_vec(1,3), unit_vec(0,3)))) == WH_basis(1,0,3)

True

In [20]:
def matrix_reshuffle(M,dim):
    matrix_reshaped = np.array(M).reshape((dim, dim, dim, dim))
    return matrix(np.array([[[[matrix_reshaped[l,j,k,i] for i in range(0,dim)] for j in range(0,dim)] for k in range(0,dim)] for l in range(0,dim)]).reshape((dim**2,dim**2)))


In [21]:
def is_dual_unitary(M,dim):
    return matrix_reshuffle(M,dim).is_unitary()


In [22]:
def matrix_partial_transpose(M,dim):
    matrix_reshaped = np.array(M).reshape((dim, dim, dim, dim))
    return matrix(np.array([[[[matrix_reshaped[k,j,i,l] for i in range(0,dim)] for j in range(0,dim)] for k in range(0,dim)] for l in range(0,dim)]).reshape((dim**2,dim**2)))


In [23]:
def is_gamma_dual_unitary(M,dim):
    return matrix_partial_transpose(M,dim).is_unitary()

In [24]:
# define sequence values for symmetric solution

def sequence_sym(k,l,x,y):
    K.<w> = CyclotomicField(3)
    if (mod(x,2) == 1 and mod(y,2) ==1):
        return K(w**mod(2*(k**2 + l**2),3))
    else:
        return K(w**mod(2*(k**2 + l**2 - (k+l+ mod(x,3)-mod(y,3))**2),3))


In [25]:
# define sequence values for sparse solution

def sequence_sparse(k,l,x,y):
    K.<w> = CyclotomicField(3)
    if (mod(x,2) == 1 and mod(y,2) ==1):
        return K(w**(2*(k**2 + l**2)))
    else:
        return K(w**(2*(k**2 + l**2 + (l + mod(x,3)-mod(y,3))**2)))


In [26]:
# define function that outputs, given a sequence seq, all sequence values as a list

def sequence_list(seq):
    list_out = []
    for values in iteration_list:
            list_out.append(seq(values[0], values[1], values[2], values[3]))
    return list_out

In [27]:
# define values of partial sequences indexed by (x,y)

def partial_sequence(x,y,seq):
    output = []
    for element in iteration_list:
        if (element[2]==x and element[3] == y):
            output.append(seq(element[0],element[1],x,y))
    return output


In [28]:
# compute sequences

sparse = np.array(sequence_list(sequence_sparse)).reshape((6,6))
symmetric  = np.array(sequence_list(sequence_sym)).reshape((6,6))

In [29]:
# define function that generates a unitary out of a given sequence

def unitary_36(seq):
    M = np.array(sequence_list(seq)).reshape((6,6))
    L = []
    for a1 in range(0,6):
        for a2 in range(0,6):
            L.append(M[a1,a2]*(WH_basis(a1,a2,6).tensor_product(WH_basis(a1,a2,6).conjugate_transpose())))
    return Matrix(CyclotomicField(3), 36,36, sum(L))



In [30]:
# compute 2-unitaries correpsonding to the sparse and the symmetric solution

U_sym = (1/36)*U_WH(6)*unitary_36(sequence_sym)*U_WH(6).conjugate_transpose() 

U_sparse = (1/36)*U_WH(6)*unitary_36(sequence_sparse)*U_WH(6).conjugate_transpose() 

In [31]:
# test 2-unitarity, symmetric

U_sym.is_unitary() and is_dual_unitary(U_sym,6) and is_gamma_dual_unitary(U_sym,6)


True

In [32]:
# test 2-unitarity, sparse 
    
U_sparse.is_unitary() and is_dual_unitary(U_sparse,6) and is_gamma_dual_unitary(U_sparse,6)

True

In [33]:
# define function that generates a unitary out of a given partial sequence

def unitary_partial_sequence(x,y,seq):
    K.<w> = CyclotomicField(3)
    M = MatrixSpace(K, 3, 3)
    L = []
    for a in range(0,3):
        for b in range(0,3):
            L.append(M(np.array(partial_sequence(x,y, seq)).reshape((3,3)))[a,b]*(WH_basis(a,b,3).tensor_product(WH_basis(a,b,3).conjugate_transpose())))
    return (1/3)*matrix(sum(L))

In [34]:
# compute partial unitary for
# (x,y) = (1,1)
        
        
USp_11 = unitary_partial_sequence(1,1,sequence_sparse)
USym_11 = unitary_partial_sequence(1,1,sequence_sym)

# check 2-unitarity

print(is_gamma_dual_unitary(USp_11,3), is_dual_unitary(USp_11,3),  USp_11.is_unitary())

print(is_gamma_dual_unitary(USym_11,3), is_dual_unitary(USym_11,3), USym_11.is_unitary())# (x,y)

True True True
True True True


In [35]:
# compute partial unitary for
# (x,y) = (0,0)

USp_00 = unitary_partial_sequence(0,0,sequence_sparse)
USym_00 = unitary_partial_sequence(0,0,sequence_sym)

# check 2-unitarity

print(is_gamma_dual_unitary(USp_00,3), is_dual_unitary(USp_00,3),  USp_00.is_unitary())

print(is_gamma_dual_unitary(USym_00,3), is_dual_unitary(USym_00,3), USym_00.is_unitary())

False True True
False True True


In [36]:
# compute partial unitary for
# (x,y) = (1,0)

USp_10 = unitary_partial_sequence(1,0,sequence_sparse)
USym_10 = unitary_partial_sequence(1,0,sequence_sym)

# check 2-unitarity

print(is_gamma_dual_unitary(USp_10,3), is_dual_unitary(USp_10,3),  USp_10.is_unitary())

print(is_gamma_dual_unitary(USym_10,3), is_dual_unitary(USym_10,3), USym_10.is_unitary())

False True True
False True True


In [37]:
# compute partial unitary for
# (x,y) = (0,1)

        
USp_01 = unitary_partial_sequence(0,1,sequence_sparse)
USym_01 = unitary_partial_sequence(0,1,sequence_sym)

# check 2-unitarity

print(is_gamma_dual_unitary(USp_01,3), is_dual_unitary(USp_01,3),  USp_01.is_unitary())

print(is_gamma_dual_unitary(USym_01,3), is_dual_unitary(USym_01,3), USym_01.is_unitary())

False True True
False True True


In [188]:
# define diagonal matrices out of partial sequences for (x,y) in {(0,0), (0,1), (1,0)}

MSp_27 = M27(diagonal_matrix(sum([partial_sequence(t[0],t[1],sequence_sparse) for t in [(0,0), (1,0), (0,1)]],[])))
MSym_27 = M27(diagonal_matrix(sum([partial_sequence(t[0],t[1],sequence_sym) for t in [(0,0), (1,0), (0,1)]],[])))


In [189]:
# define identity matrices

Id3 = identity_matrix(3)
Id9 = identity_matrix(9)



In [195]:
# test

Matrix(K, 27, 27, USp_00.block_sum(USp_10.block_sum(USp_01))) == (1/3)*Id3.tensor_product(U_WH(3))*MSp_27*Id3.tensor_product(U_WH(3)).conjugate_transpose()

True

In [65]:
# define list of bell states

Bell_basis_list = [V4(vector(np.kron(unit_vec(0,2), unit_vec(0,2))+np.kron(unit_vec(1,2), unit_vec(1,2)))), V4(vector(np.kron(unit_vec(0,2), unit_vec(0,2)) - np.kron(unit_vec(1,2), unit_vec(1,2)))), V4(vector(np.kron(unit_vec(0,2), unit_vec(1,2))+np.kron(unit_vec(1,2),unit_vec(0,2))))
             , V4(vector(np.kron(unit_vec(0,2), unit_vec(1,2))-np.kron(unit_vec(1,2), unit_vec(0,2))))]



In [180]:
# define matrix V (in the dissertation: B)

dic = {(0,1): 2,(1,0) : 1, (0,0): 3, (1,1): 0}

l = []
for a in [(1,1),(0,1),(1,0)]:
        x= mod(a[0]-a[1],3)
        l.append((1/sqrt(2))*Bell_basis_list[dic[a]].tensor_product(V3(vector(unit_vec(x, 3))).conjugate_transpose()))



V = Matrix( sum(l))

V

[ 1/2*sqrt(2)  1/2*sqrt(2)            0]
[           0            0  1/2*sqrt(2)]
[           0            0  1/2*sqrt(2)]
[ 1/2*sqrt(2) -1/2*sqrt(2)            0]

In [184]:
# write out decomposition for sparse solution

USp_1 = (1/2)*USp_11.tensor_product(Bell_basis_list[3].tensor_product(Bell_basis_list[3].conjugate_transpose()))

USp_2 = (1/3)*Matrix(K, 27, 27, (U_WH(3).tensor_product(Id3)*MSp_27*((U_WH(3).tensor_product(Id3)).conjugate_transpose())))

#USp_2 = 

USp_lambda = M36(USp_1 + Id9.tensor_product(V)*USp_2*((Id9).tensor_product(V.conjugate_transpose())))


In [196]:
# write out decomposition for symmetric solution

USym_1 = (1/2)*USym_11.tensor_product(Bell_basis_list[3].tensor_product(Bell_basis_list[3].conjugate_transpose()))


USym_2 = (1/3)*U_WH(3).tensor_product(identity_matrix(3))*MSym_27*(U_WH(3).tensor_product(identity_matrix(3))).conjugate_transpose()
#Matrix(K, 27, 27,USym_00.block_sum(USym_10.block_sum(USym_01)))

USym_lambda = M36(USym_1 + Id9.tensor_product(V)*USym_2*(Id9.tensor_product(V)).conjugate_transpose())


In [185]:
print((USp_lambda*USp_lambda.conjugate_transpose()).str())

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0

In [197]:
print((USym_lambda*USym_lambda.conjugate_transpose()).str())

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0

In [95]:
# define WH basis as list 

Basis = flatten([[powers_of_X(6)[i]*powers_of_Z(6)[j] for i in range(0,6)] for j in range(0,6)])

In [57]:
# define local subalgebras 

Id6= identity_matrix(6)

R = [M36(Id6.tensor_product(element)) for element in Basis]

L = [M36(element.tensor_product(Id6)) for element in Basis]

LocalSubal = [L,R]

In [96]:
def is_quasi_orth(A,B):
    for V in A:
            for W in B:
                K = V*W
                if (K.trace())/(K.nrows()) != ((V.trace())*(W.trace()))/((V.nrows())*(W.nrows())):
                    return false
    return true

is_quasi_orth(R,L)

True

In [92]:
# define a function that computes the conjugation of a unitary U on the local subalgebra L

def AL(U):
    return [M36(U*element*U.conjuggate_transpose()) for element in L]

AL(U_sparse)

[36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cyclotomic Field of order 3 and degree 2,
 36 x 36 dense matrix over Cycloto

In [101]:
# define a function that computes the conjugation of a unitary U on the local subalgebra R

def AR(U):
    return [M36(U*element*U.conjugate_transpose()) for element in R]

In [97]:
def is_delocalised(A,O):
    for N in O:
        if is_quasi_orth(A,N) == false:
            return false
    return true


In [98]:
is_delocalised(AL(U_sparse), LocalSubal)

True

In [102]:
is_delocalised(AR(U_sparse), LocalSubal)

True

In [104]:
# define a basis dictionary

Basis_dic = {0 : "Id6", 1 : "Id6 Z", 2: "Id6 Z^2", 3: "Id6 Z^3", 4:"Id6 Z^4", 5: "Id6 Z^5", 6: "X Id6", 7 : "X Z", 8: "X Z^2",9 : "X Z^3", 10 : "X Z^4", 11 : "X Z^5", 12 : "X^2 Id6", 13 : "X^2 Z" , 14: "X^2 Z^2", 15: "X^2 Z^3", 16: "X^2 Z^4", 17: "X^2 Z^5",18: "X^3 Id6", 19: "X^3 Z", 20: "X^3 Z^2", 21: "X^3 Z^3", 22: "X^3 Z^4", 23: "X^3 Z^5",24: "X^4 Id6",25: "X^4 Z", 26: "X^4 Z^2", 27: "X^4 Z^3", 28: "X^4 Z^4", 29: "X^4 Z^5",30: "X^5 Id6", 31: "X^5 Z",32: "X^5 Z^2", 33: "X^5 Z^3", 34: "X^5 Z^4", 35: "X^5 Z^5"}

In [119]:
# check if rotated subalgebra AL can be expressed through WH-operators

AL_dic = []

for M in AL(U_sparse):
    for j in range(0,len(Basis)):
        for k in range(0,len(Basis)):
            #print(Matrix(Basis[j].tensor_product(Basis[k])), ' ', [j,k], '\n')
            for m in range(0,6):
                if M == (f**m)*Basis[j].tensor_product(Basis[k]):
                    AL_dic.append([m,Basis_dic[j], Basis_dic[k]])

In [124]:
for M in AL_dic:
    print("m:", M[0], M[1],"x", M[2], "\n")

m: 0 Id6 x Id6 

m: 4 X^4 Z^4 x Id6 Z^2 

m: 4 X^2 Z^2 x Id6 Z^4 



In [122]:
# check if rotated subalgebra AR can be expressed through WH-operators

AR_dic = []

for M in AR(U_sparse):
    for j in range(0,len(Basis)):
        for k in range(0,len(Basis)):
            #print(Matrix(Basis[j].tensor_product(Basis[k])), ' ', [j,k], '\n')
            for l in range(0,6):
                if M == (f**m)*Basis[j].tensor_product(Basis[k]):
                    AR_dic.append([m,Basis_dic[j], Basis_dic[k]])

In [125]:
for M in AR_dic:
    print("m:", M[0], M[1],"x", M[2], "\n")